In [1]:
import json
from pathlib import Path

from findingmodel import FindingInfo, FindingModelFull
from findingmodel.common import model_file_name
from findingmodel.index_code import IndexCode
from findingmodel.tools import (
    add_ids_to_finding_model,
    add_standard_codes_to_finding_model,
    create_finding_model_stub_from_finding_info,
)


In [ ]:
json_file = Path("data") / "chest_gamuts.json"
gamuts_data = json.loads(json_file.read_text())

In [3]:
gamuts_data = list(gamuts_data.values())
len(gamuts_data)

260

In [12]:
def stub_from_gamut(
    gamuts_id: str,
    name: str,
    seen_on_modalities: list[str],
    description: str,
    synonyms: list[str],
    category: str,
    subcategories: list[str],
    region: str,
    **extra_args: dict,
) -> FindingModelFull:
    info = FindingInfo(name=name, description=description, synonyms=synonyms)
    fm = create_finding_model_stub_from_finding_info(info)
    tags = [region, *seen_on_modalities, *subcategories, category]
    fm.tags = tags
    gamut_code = IndexCode(system="GAMUTS", code=gamuts_id, display=name)
    fm_ids = add_ids_to_finding_model(fm, source="GMTS")
    fm_ids.index_codes = [gamut_code]
    # Make a string of the gamuts_id zero-padded to 6 digits
    gamuts_id_string = gamuts_id.zfill(6)
    fm_ids.oifm_id = fm_ids.oifm_id[:-6] + gamuts_id_string
    add_standard_codes_to_finding_model(fm_ids)
    return fm_ids


In [13]:
models = [stub_from_gamut(**gamut, region="chest") for gamut in gamuts_data]
len(models)

260

In [14]:
def write_model(fm: FindingModelFull) -> int:
    file_out = Path("data") / "defs" / model_file_name(fm.name)
    return file_out.write_text(
        fm.model_dump_json(indent=2, exclude_none=True),
        encoding="utf-8",
    )

In [15]:
for model in models:
    write_model(model)